In [1]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"

from os import listdir
from os.path import isfile, join
import sys
stylegan2_path = './stylegan2-ada-pytorch'
sys.path.append(stylegan2_path)

import pickle
import dnnlib
import click
import legacy
import torch
import numpy as np
from PIL import Image
from scipy.spatial.distance import cosine
import torch.nn.functional as F

In [2]:
goal_attr_change = 2.197 # We want the sigmoid output in the range of 0.1..0.9. I
# in case of a sigmoid output of 0.5 we only want to change the target attribute for 0.4 ~ 2.197 for the sigmoid input
target_attr = 31
stepsize = 0.5 
break_delta=0.01
max_iter=50

n_regressor_predictions = 40 # the regressor is pretrained on CelebA and predicts 40 face attributes
device = 'cuda:0'
batch_size = 1
truncation_psi = 0.5
noise_mode = 'const'
network_pkl =  './pretrained_models/ffhq.pkl'

with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # stylegan2-generator

label = torch.zeros([1, G.c_dim], device=device)

def initialize_model():
    from facenet_pytorch import MTCNN, InceptionResnetV1
    resnet = InceptionResnetV1(pretrained='vggface2').to(device).eval()
    return resnet

face_rec = initialize_model()

file_to_read = open("./pretrained_models/resnet_092_all_attr_5_epochs.pkl", "rb")
regressor = pickle.load(file_to_read)
file_to_read.close()
regressor = regressor.to(device)
regressor.eval()

def get_pred_from_img(w, target_attr):
    img = G.synthesis(w, noise_mode=noise_mode)
    img = F.interpolate(img, size=256) # reshape the image for the face_rec and regressor
    pred = regressor(img).detach().cpu().numpy()
    return pred[0, target_attr]

def get_face_embed(img):
    img_org_np = np.uint8(np.clip(((img.detach().cpu().numpy() + 1) / 2.0) * 255, 0, 255))
    org = Image.fromarray(np.transpose(img_org_np[0], (1,2,0)))
    reshaped_org = org.resize((160, 160))
    reshaped_org = torch.Tensor(np.transpose(np.array(reshaped_org), (2,0,1))).to(device).unsqueeze(0)
    embed_org = face_rec(reshaped_org)
    return embed_org

# Evaluating in Z-Space for Larsen et al.

In [3]:
def get_scaling_factor_z(l_vec1, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter):
    i = 0
    scaling_direction_flag = 0

    while(True):
        target_attr_change_list = []
        for j in range(batch_size):
            z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
            w = G.mapping(z,label, truncation_psi=truncation_psi)
            pred_orig = get_pred_from_img(w, target_attr)
            
            if pred_orig > 0.0:
                z_new = z - (l_vec1*scaling_factor) # differs here compared to w-Space
                w_new = G.mapping(z_new,label, truncation_psi=truncation_psi)
                target_attr_change = get_pred_from_img(w_new, target_attr) - pred_orig
            else:
                z_new = z + (l_vec1*scaling_factor) # differs here compared to w-Space
                w_new = G.mapping(z_new,label, truncation_psi=truncation_psi)
                target_attr_change = get_pred_from_img(w_new, target_attr) - pred_orig

            target_attr_change_list.append(target_attr_change)

        avg_attr_delta = np.abs(np.array(target_attr_change_list)).mean()

        if np.abs(avg_attr_delta - goal_attr_change) < break_delta:
            print("delta reached: ", avg_attr_delta - goal_attr_change)
            return scaling_factor
        else:              
            if avg_attr_delta < goal_attr_change:
                scaling_factor += stepsize
                if scaling_direction_flag == -1:
                    stepsize = stepsize/2
                scaling_direction_flag = 1
            elif avg_attr_delta > goal_attr_change:
                scaling_factor -= stepsize
                if scaling_direction_flag == 1:
                    stepsize = stepsize/2
                scaling_direction_flag = -1

        i+=1
        if i > max_iter:
            return scaling_factor


def eval_attr_vec_z(l_vec, target_attr, num_samples):
    pred_diff_list = []
    embedding_distance_list = []
    attr_pres_list = []

    for j in range(num_samples):
        z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
        w = G.mapping(z,label, truncation_psi=truncation_psi)
        img_orig = G.synthesis(w, noise_mode=noise_mode)
        img_orig = F.interpolate(img_orig, size=256) # reshape the image for the face_rec and regressor
        pred_orig = regressor(img_orig).detach().cpu().numpy()
        embedding_orig = get_face_embed(img_orig)
        random_scaling = torch.from_numpy(np.random.RandomState(j).rand(1, 1)).to(device) 

        if pred_orig[0, target_attr] > 0.0:
            z_new = z - (l_vec*random_scaling) # differs here compared to w-Space
            w_new = G.mapping(z_new,label, truncation_psi=truncation_psi)
            img_shifted = G.synthesis(w_new, noise_mode=noise_mode)
            
        else:
            z_new = z + (l_vec*random_scaling) # differs here compared to w-Space
            w_new = G.mapping(z_new,label, truncation_psi=truncation_psi)
            img_shifted = G.synthesis(w_new, noise_mode=noise_mode)

        img_shifted = F.interpolate(img_shifted, size=256) # reshape the image for the face_rec and regressor
        pred_shifted = regressor(img_shifted).detach().cpu().numpy()

        # cosine distance
        embedding_shifted = get_face_embed(img_shifted)
        embedding_distance = cosine(embedding_orig.detach().cpu().numpy(), embedding_shifted.detach().cpu().numpy())
        embedding_distance_list.append(embedding_distance)
        
        # attribute change
        pred_diff_list.append(np.abs(pred_shifted[0, target_attr] - pred_orig[0, target_attr] ))

        # attribute preservation
        other_pred_org = np.hstack([pred_orig[:, :int(target_attr)], pred_orig[:, int(target_attr + 1):]])
        other_pred_shifted = np.hstack([pred_shifted[:, :int(target_attr)], pred_shifted[:, int(target_attr + 1):]])
        attr_preservation = np.mean(np.abs(other_pred_org - other_pred_shifted))
        attr_pres_list.append(attr_preservation)
        
    target_change = np.array(pred_diff_list).mean()
    attr_dist = np.array(attr_pres_list).mean()
    arcf_dist = np.array(embedding_distance).mean()
    target_change_std = np.array(pred_diff_list).std()
    attr_dist_std = np.array(attr_pres_list).std()
    arcf_dist_std = np.array(embedding_distance_list).std()
    return target_change, target_change_std, attr_dist, attr_dist_std, arcf_dist, arcf_dist_std
    

def scale_and_eval_z(attr_vec, target_attr):
    scaling_factor = 1
    for batch_size in [8, 32]: # habe die 1000 für einen speedup entfernt
        scaling_factor = get_scaling_factor_z(attr_vec, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter)
        print("bs: ", batch_size, "scaling: ", scaling_factor)

        attribute_change, attr_ch_std, attribute_preservation, attr_pres_std, similarity_metric, sim_std = eval_attr_vec_z(scaling_factor*attr_vec, target_attr, 1000)
        print("attribute_change: ", attribute_change, attr_ch_std)
        print("attribute_preservation: ", attribute_preservation, attr_pres_std)
        print("similarity_metric: ", similarity_metric, sim_std)
        print("-------------------------------")

l_vec_smile_z = torch.load("./attribute_vectors/l_vec_larsen_smile_z.pt", map_location=device)
l_vec_hair9_z = torch.load("./attribute_vectors/l_vec_larsen_hair9_z.pt", map_location=device)

print("l_vec_larsen_hair9_z.pt")
scale_and_eval_z(l_vec_hair9_z, 9) # 9 is haircolor
print("-------------------------")

print("l_vec_larsen_smile_z.pt")
scale_and_eval_z(l_vec_smile_z, 31) # 31 is smile
print("-------------------------")

l_vec_larsen_hair9_z.pt
delta reached:  0.001062419891357358
bs:  8 scaling:  0.48046875
attribute_change:  0.6855873 0.84384847
attribute_preservation:  0.40567508 0.3047385
similarity_metric:  0.009771287441253662 0.005382404291572461
-------------------------------
delta reached:  -0.0022244186401367827
bs:  32 scaling:  0.66015625
attribute_change:  0.98020506 1.1740159
attribute_preservation:  0.5348706 0.39615202
similarity_metric:  0.013956308364868164 0.008608207461314596
-------------------------------
-------------------------
l_vec_larsen_smile_z.pt
delta reached:  -0.00976798248291022
bs:  8 scaling:  0.6484375
attribute_change:  1.0405023 1.017158
attribute_preservation:  0.27644825 0.19848527
similarity_metric:  0.007528960704803467 0.0024266278456064293
-------------------------------
delta reached:  0.005889442443847592
bs:  32 scaling:  0.546875
attribute_change:  0.86475414 0.86227775
attribute_preservation:  0.238849 0.172537
similarity_metric:  0.00617372989654541 0

# Evaluating in W-Space

In [4]:
def get_scaling_factor(l_vec1, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter):
    i = 0
    scaling_direction_flag = 0

    while(True):
        target_attr_change_list = []
        for j in range(batch_size):
            z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
            w = G.mapping(z,label, truncation_psi=truncation_psi)
            pred_orig = get_pred_from_img(w, target_attr)

            if pred_orig > 0.0:
                target_attr_change = get_pred_from_img(w-(l_vec1*scaling_factor), target_attr) - pred_orig
            else:
                target_attr_change = get_pred_from_img(w+(l_vec1*scaling_factor), target_attr) - pred_orig

            target_attr_change_list.append(target_attr_change)

        avg_attr_delta = np.abs(np.array(target_attr_change_list)).mean()

        if np.abs(avg_attr_delta - goal_attr_change) < break_delta:
            print("delta reached: ", avg_attr_delta - goal_attr_change)
            return scaling_factor
            
        else:              
            if avg_attr_delta < goal_attr_change:
                scaling_factor += stepsize
                if scaling_direction_flag == -1:
                    stepsize = stepsize/2
                scaling_direction_flag = 1
            elif avg_attr_delta > goal_attr_change:
                scaling_factor -= stepsize
                if scaling_direction_flag == 1:
                    stepsize = stepsize/2
                scaling_direction_flag = -1

        i+=1
        if i > max_iter:
            return scaling_factor
            #break


def eval_attr_vec(l_vec, target_attr, num_samples):
    pred_diff_list = []
    embedding_distance_list = []
    attr_pres_list = []

    for j in range(num_samples):
        z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
        w = G.mapping(z,label, truncation_psi=truncation_psi)
        img_orig = G.synthesis(w, noise_mode=noise_mode)
        img_orig = F.interpolate(img_orig, size=256) # reshape the image for the face_rec and regressor
        pred_orig = regressor(img_orig).detach().cpu().numpy()
        embedding_orig = get_face_embed(img_orig)
        random_scaling = torch.from_numpy(np.random.RandomState(j).rand(1, 1)).to(device) # ÄNDERUNG fürs scaling!!!

        if pred_orig[0, target_attr] > 0.0:
            img_shifted = G.synthesis(w-(l_vec*random_scaling), noise_mode=noise_mode)
        else:
            img_shifted = G.synthesis(w+(l_vec*random_scaling), noise_mode=noise_mode)

        img_shifted = F.interpolate(img_shifted, size=256) # reshape the image for the face_rec and regressor
        pred_shifted = regressor(img_shifted).detach().cpu().numpy()

        # cosine distance
        embedding_shifted = get_face_embed(img_shifted)
        embedding_distance = cosine(embedding_orig.detach().cpu().numpy(), embedding_shifted.detach().cpu().numpy())
        embedding_distance_list.append(embedding_distance)
        
        # attribute change
        pred_diff_list.append(np.abs(pred_shifted[0, target_attr] - pred_orig[0, target_attr] ))

        # attribute preservation
        other_pred_org = np.hstack([pred_orig[:, :int(target_attr)], pred_orig[:, int(target_attr + 1):]])
        other_pred_shifted = np.hstack([pred_shifted[:, :int(target_attr)], pred_shifted[:, int(target_attr + 1):]])
        attr_preservation = np.mean(np.abs(other_pred_org - other_pred_shifted))
        attr_pres_list.append(attr_preservation)
        
    target_change = np.array(pred_diff_list).mean()
    attr_dist = np.array(attr_pres_list).mean()
    arcf_dist = np.array(embedding_distance).mean()
    target_change_std = np.array(pred_diff_list).std()
    attr_dist_std = np.array(attr_pres_list).std()
    arcf_dist_std = np.array(embedding_distance_list).std()
    return target_change, target_change_std, attr_dist, attr_dist_std, arcf_dist, arcf_dist_std



goal_attr_change = 2.197 # in a case of an initial attribute vector of 0.5 this changes the sigmoid value by 0.4
target_attr = 31
stepsize = 0.5 
break_delta=0.01
max_iter=50

def scale_and_eval(attr_vec, target_attr):
    scaling_factor = 1.0
    for batch_size in [8, 32]: 
        scaling_factor = get_scaling_factor(attr_vec, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter)
        print("bs: ", batch_size, "scaling: ", scaling_factor)

        attribute_change, attr_change_std, attribute_preservation, attr_pres_std, similarity_metric, sim_metric_std = eval_attr_vec(scaling_factor*attr_vec, target_attr, batch_size)
        print("attribute_change: ", attribute_change, attr_change_std)
        print("attribute_preservation: ", attribute_preservation, attr_pres_std)
        print("similarity_metric: ", similarity_metric, sim_metric_std)

        print("-------------------------------")

In [5]:
print("l_vec_larsen_hair9_w")
attr_vec = torch.load("./attribute_vectors/l_vec_larsen_hair9_w.pt", map_location=device)
scale_and_eval(attr_vec, 9) 

print("l_vec_larsen_smile_w")
attr_vec = torch.load("./attribute_vectors/l_vec_larsen_smile_w.pt", map_location=device)
scale_and_eval(attr_vec, 31)

l_vec_larsen_hair9_w
delta reached:  -0.0025624961853027983
bs:  8 scaling:  0.375
attribute_change:  1.1518537 1.1955967
attribute_preservation:  0.5238164 0.42857862
similarity_metric:  9.191036224365234e-05 0.00352196014370181
-------------------------------
delta reached:  -0.004579517364502017
bs:  32 scaling:  0.44921875
attribute_change:  0.8327917 0.8929268
attribute_preservation:  0.52067816 0.3828484
similarity_metric:  0.0010453462600708008 0.004496470586099009
-------------------------------
l_vec_larsen_smile_w
delta reached:  -0.00984284591674811
bs:  8 scaling:  0.30078125
attribute_change:  1.2729964 0.99157846
attribute_preservation:  0.28808972 0.16952616
similarity_metric:  6.9141387939453125e-06 0.0007238071992392287
-------------------------------
delta reached:  0.009894874572753842
bs:  32 scaling:  0.2890625
attribute_change:  1.1427507 1.0444833
attribute_preservation:  0.21359012 0.14280128
similarity_metric:  4.6372413635253906e-05 0.0005941820974235821
-----

In [10]:
print("l_vec_shen_hair9_w.npy")
attr_vec = torch.from_numpy(np.load("./attribute_vectors/l_vec_shen_hair9_w.npy")).to(device)
scale_and_eval(attr_vec, 9)

print("shen_smiling_w.npy")
attr_vec = torch.from_numpy(np.load("./attribute_vectors/l_vec_shen_smile_w.npy")).to(device)
scale_and_eval(attr_vec, 31)

l_vec_shen_hair9_w.npy


TypeError: from_numpy() takes no keyword arguments

In [8]:
print("zhuang_hair_col_9_w")
attr_vec = torch.load("./attribute_vectors/l_vec_zhuang_hair9_w.pt", map_location=device)
scale_and_eval(attr_vec, 9)

print("zhuang_smiling_w")
attr_vec = torch.load("./attribute_vectors/l_vec_zhuang_smile_w.pt", map_location=device)
scale_and_eval(attr_vec, 31)

zhuang_hair_col_9_w
delta reached:  -0.004005882263183658
bs:  8 scaling:  0.201171875
attribute_change:  1.1513443 1.145913
attribute_preservation:  0.20349863 0.14837822
similarity_metric:  5.620718002319336e-05 0.004495562677234512
-------------------------------
delta reached:  -0.0001530380249024077
bs:  32 scaling:  0.248046875
attribute_change:  0.8933507 0.8757463
attribute_preservation:  0.2437277 0.15271558
similarity_metric:  0.0005311369895935059 0.004209498371201054
-------------------------------
zhuang_smiling_w
delta reached:  -0.0051264495849610014
bs:  8 scaling:  0.189453125
attribute_change:  1.246439 0.964671
attribute_preservation:  0.24842453 0.15758057
similarity_metric:  3.874301910400391e-06 0.0006259568222167492
-------------------------------
bs:  32 scaling:  -25.310546875
attribute_change:  4.58639 2.7494304
attribute_preservation:  2.8290644 1.0207511
similarity_metric:  0.04728376865386963 0.05113052201497457
-------------------------------


In [11]:
print("l_vec_LS-StyleEdit_hair9_w")
attr_vec = torch.load("./attribute_vectors/l_vec_LS-StyleEdit_hair9_w.pt", map_location=device)
scale_and_eval(attr_vec, 9)

print("l_vec_LS-StyleEdit_smile_w")
attr_vec = torch.load("./attribute_vectors/l_vec_LS-StyleEdit_smile_w.pt", map_location=device)
scale_and_eval(attr_vec, 31)

l_vec_LS-StyleEdit_hair9_w
delta reached:  0.0017919807434081392
bs:  8 scaling:  0.1220703125
attribute_change:  1.1609406 1.1405779
attribute_preservation:  0.253304 0.13630417
similarity_metric:  4.935264587402344e-05 0.005647686298331719
-------------------------------
delta reached:  0.007776287078857358
bs:  32 scaling:  0.1611328125
attribute_change:  0.8941095 0.917514
attribute_preservation:  0.30026308 0.17880459
similarity_metric:  0.0005733370780944824 0.005830954863870075
-------------------------------
l_vec_LS-StyleEdit_smile_w
delta reached:  -0.007851812362670962
bs:  8 scaling:  0.34765625
attribute_change:  1.2244022 0.9263094
attribute_preservation:  0.24829997 0.1602671
similarity_metric:  3.814697265625e-06 0.0005230662642874221
-------------------------------
delta reached:  -0.009271595001220767
bs:  32 scaling:  0.32421875
attribute_change:  1.1481775 1.0420691
attribute_preservation:  0.18176383 0.13103719
similarity_metric:  1.9609928131103516e-05 0.000296666

# Evaluating in S-Space

In [6]:
%pip uninstall torch-utils

Uninstalling torch-utils-0.1.2:
  Would remove:
    /home/hermiod/projects/tests/stylegan3/venv_faceon/lib/python3.8/site-packages/torch_utils-0.1.2.dist-info/*
    /home/hermiod/projects/tests/stylegan3/venv_faceon/lib/python3.8/site-packages/torch_utils/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [9]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"

from os import listdir
from os.path import isfile, join
from natsort import natsorted
import sys
library_path = os.path.abspath('./tmp_styleclip')

# Add the library path to sys.path
sys.path.insert(0, library_path)

import pickle
import dnnlib
import click
from tmp_styleclip import legacy
import torch
import numpy as np
from PIL import Image
from scipy.spatial.distance import cosine
import torch.nn.functional as F
from tmp_styleclip.training.networks import SynthesisNetwork,SynthesisBlock,SynthesisLayer,ToRGBLayer
from tmp_styleclip.torch_utils import misc
import types

n_regressor_predictions = 40 # the regressor is pretrained on CelebA and predicts 40 face attributes
device = 'cuda:0'
batch_size = 1
truncation_psi = 0.5
noise_mode = 'const'
network_pkl =  '../../pretrained_models/ffhq.pkl'

ImportError: cannot import name 'misc' from 'torch_utils' (/home/hermiod/projects/tests/stylegan3/venv_faceon/lib/python3.8/site-packages/torch_utils/__init__.py)

In [1]:
import types
from tmp_styleclip.training.networks import SynthesisNetwork,SynthesisBlock,SynthesisLayer,ToRGBLayer

def LoadModel(network_pkl,device):
    with dnnlib.util.open_url(network_pkl) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore
    
    G.synthesis.forward=types.MethodType(SynthesisNetwork.forward,G.synthesis)
    G.synthesis.W2S=types.MethodType(SynthesisNetwork.W2S,G.synthesis)
    
    for res in G.synthesis.block_resolutions:
        block = getattr(G.synthesis, f'b{res}')
        # print(block)
        block.forward=types.MethodType(SynthesisBlock.forward,block)
        
        if res!=4:
            layer=block.conv0
            layer.forward=types.MethodType(SynthesisLayer.forward,layer)
            layer.name='conv0_resolution_'+str(res)
            
        layer=block.conv1
        layer.forward=types.MethodType(SynthesisLayer.forward,layer)
        layer.name='conv1_resolution_'+str(res)
        
        layer=block.torgb
        layer.forward=types.MethodType(ToRGBLayer.forward,layer)
        layer.name='toRGB_resolution_'+str(res)
        
    
    return G

G = LoadModel(network_pkl, device)   
label = torch.zeros([1, G.c_dim], device=device)

def add_attr_vec(encoded_styles, attr_vec, epsilon):
    new_attr_vec = {}
    i=0
    for key in encoded_styles.keys():
        new_attr_vec[key] = encoded_styles[key] + attr_vec[0:, i:i+encoded_styles[key].shape[1]]*epsilon
        i+=encoded_styles[key].shape[1]
    return new_attr_vec

ImportError: cannot import name 'misc' from 'torch_utils' (/home/hermiod/projects/tests/stylegan3/venv_faceon/lib/python3.8/site-packages/torch_utils/__init__.py)

In [17]:
def eval_attr_vec_s(l_vec, target_attr, num_samples):
    pred_diff_list = []
    embedding_distance_list = []
    attr_pres_list = []

    for j in range(num_samples):
        z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
        w = G.mapping(z,label, truncation_psi=truncation_psi)
        img_orig = G.synthesis(w, noise_mode=noise_mode)
        img_orig = F.interpolate(img_orig, size=256) # reshape the image for the face_rec and regressor
        pred_orig = regressor(img_orig).detach().cpu().numpy()
        embedding_orig = get_face_embed(img_orig)
        random_scaling = torch.from_numpy(np.random.RandomState(j).rand(1, 1)).to(device)
        #random_scaling = torch.Tensor([1.0]).to(device)

        if pred_orig[0, target_attr] > 0.0:
            #img_shifted = G.synthesis(w-(l_vec*random_scaling), noise_mode=noise_mode)
            s=G.synthesis.W2S(w)
            s_new = add_attr_vec(s, l_vec, -random_scaling)
            img_shifted = G.synthesis(None, encoded_styles=s_new,noise_mode='const')
            
        else:
            #img_shifted = G.synthesis(w+(l_vec*random_scaling), noise_mode=noise_mode)
            s=G.synthesis.W2S(w)
            s_new = add_attr_vec(s, l_vec, random_scaling)
            img_shifted = G.synthesis(None, encoded_styles=s_new,noise_mode='const')

        img_shifted = F.interpolate(img_shifted, size=256) # reshape the image for the face_rec and regressor
        pred_shifted = regressor(img_shifted).detach().cpu().numpy()

        # cosine distance
        embedding_shifted = get_face_embed(img_shifted)
        embedding_distance = cosine(embedding_orig.detach().cpu().numpy(), embedding_shifted.detach().cpu().numpy())
        embedding_distance_list.append(embedding_distance)
        
        # attribute change
        pred_diff_list.append(np.abs(pred_shifted[0, target_attr] - pred_orig[0, target_attr] ))

        # attribute preservation
        other_pred_org = np.hstack([pred_orig[:, :int(target_attr)], pred_orig[:, int(target_attr + 1):]])
        other_pred_shifted = np.hstack([pred_shifted[:, :int(target_attr)], pred_shifted[:, int(target_attr + 1):]])
        attr_preservation = np.mean(np.abs(other_pred_org - other_pred_shifted))
        attr_pres_list.append(attr_preservation)
        
    target_change = np.array(pred_diff_list).mean()
    attr_dist = np.array(attr_pres_list).mean()
    arcf_dist = np.array(embedding_distance).mean()
    target_change_std = np.array(pred_diff_list).std()
    attr_dist_std = np.array(attr_pres_list).std()
    arcf_dist_std = np.array(embedding_distance_list).std()
    return target_change, target_change_std, attr_dist, attr_dist_std, arcf_dist, arcf_dist_std

In [18]:
def get_pred_from_img_s(s, target_attr):
    img = G.synthesis(None, encoded_styles=s,noise_mode='const')
    img = F.interpolate(img, size=256) # reshape the image for the face_rec and regressor
    pred = regressor(img).detach().cpu().numpy()
    return pred[0, target_attr]

def get_scaling_factor_s(l_vec1, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter):
    i = 0
    scaling_direction_flag = 0

    while(True):
        target_attr_change_list = []
        for j in range(batch_size):
            z = torch.from_numpy(np.random.RandomState(j).randn(1, 512)).to(device)
            w = G.mapping(z,label, truncation_psi=truncation_psi)
            s=G.synthesis.W2S(w)
            pred_orig = get_pred_from_img_s(s, target_attr)

            if pred_orig > 0.0:
                target_attr_change = get_pred_from_img_s(add_attr_vec(s, l_vec1, -scaling_factor), target_attr) - pred_orig
            else:
                target_attr_change = get_pred_from_img_s(add_attr_vec(s, l_vec1, scaling_factor), target_attr) - pred_orig

            target_attr_change_list.append(target_attr_change)

        avg_attr_delta = np.abs(np.array(target_attr_change_list)).mean()
        
        if np.abs(avg_attr_delta - goal_attr_change) < break_delta:
            #print("delta reached: ", avg_attr_delta - goal_attr_change)
            return scaling_factor, avg_attr_delta
            #break
        else:              
            if avg_attr_delta < goal_attr_change:
                scaling_factor += stepsize
                #print("positive", avg_attr_delta, goal_attr_change)
                if scaling_direction_flag == -1:
                    stepsize = stepsize/2
                scaling_direction_flag = 1
            elif avg_attr_delta > goal_attr_change:
                scaling_factor -= stepsize
                #print("negative")
                if scaling_direction_flag == 1:
                    stepsize = stepsize/2
                scaling_direction_flag = -1

        #print(avg_attr_delta, goal_attr_change, scaling_factor)

        i+=1
        if i > max_iter:
            return scaling_factor, avg_attr_delta
            #break

In [19]:
def scale_and_eval_s(attr_vec, target_attr):
    scaling_factor = 1
    for batch_size in [8, 32]: # habe die 1000 für einen speedup entfernt
        scaling_factor = get_scaling_factor_s(attr_vec, goal_attr_change, target_attr, batch_size, scaling_factor, stepsize, break_delta, max_iter)
        print("bs: ", batch_size, "scaling: ", scaling_factor)

        attribute_change, attr_ch_std, attribute_preservation, attr_pres_std, similarity_metric, sim_std = eval_attr_vec_z(scaling_factor*attr_vec, target_attr, 1000)
        print("attribute_change: ", attribute_change, attr_ch_std)
        print("attribute_preservation: ", attribute_preservation, attr_pres_std)
        print("similarity_metric: ", similarity_metric, sim_std)
        print("-------------------------------")

print("l_vec_stylemc_blonde.npy")
attr_vec = torch.from_numpy(np.load("./attribute_vectors/l_vec_stylemc_blonde.npy")).to(device)
scale_and_eval_s(attr_vec, 9)

print("l_vec_stylemc_smile.npy")
attr_vec = torch.from_numpy(np.load("./attribute_vectors/l_vec_stylemc_smile.npy")).to(device)
scale_and_eval_s(attr_vec, 31)

l_vec_stylemc_blonde.npy


RuntimeError: The size of tensor a (256) must match the size of tensor b (512) at non-singleton dimension 2

In [ ]:
print("l_vec_LS-StyleEdit_hair9_w")
attr_vec = torch.load("./attribute_vectors/l_vec_LS-StyleEdit_hair9_s.pt", map_location=device)
scale_and_eval_s(attr_vec, 9)

print("l_vec_LS-StyleEdit_smile_w")
attr_vec = torch.load("./attribute_vectors/l_vec_LS-StyleEdit_smile_s.pt", map_location=device)
scale_and_eval_s(attr_vec, 31)